In [9]:
# https://quantumcomputing.stackexchange.com/questions/24050/how-to-implement-a-exponential-of-a-hamiltonian-but-non-unitary-matrix-in-qisk

# control gate: https://qiskit.org/documentation/stubs/qiskit.circuit.ControlledGate.html

# circuit https://www.nature.com/articles/s41598-022-17660-8
import numpy as np

from qiskit.circuit import ControlledGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute, Aer, transpile
from qiskit.quantum_info.operators import Operator
from qiskit.extensions import UnitaryGate
from qiskit.tools.visualization import plot_histogram

from qiskit.synthesis import MatrixExponential
from qiskit.quantum_info import Operator
from qiskit.quantum_info import SparsePauliOp
from qiskit.opflow.list_ops import SummedOp
from qiskit.circuit import Parameter
from qiskit.opflow import I, X, Y, Z, H, CX, Zero, ListOp, PauliExpectation, PauliTrotterEvolution, CircuitSampler, MatrixEvolution, Suzuki, PauliSumOp

In [5]:
# A = np.array([
#     [11, 5, -1, -1],
#     [5, 11, 1, 1],
#     [-1, 1, 11, -5],
#     [-1, 1, -5, 11],
#     ]).astype('complex')

# A = np.array([[ 0.6201803 , -0.0884858 ,  0.06018728, -0.47339672],
#               [-0.0884858 ,  0.97938565,  0.0140217 , -0.11028624],
#               [ 0.06018728,  0.0140217 ,  0.99046256,  0.07501575],
#               [-0.47339672, -0.11028624,  0.07501575,  0.4099715 ]]).astype('complex')
A = np.array([[ 1.20061653,  0.37361167,  0.2394791 ,  0.39870151],
              [ 0.37361167,  2.43936584,  0.75247403, -0.17244919],
              [ 0.2394791 ,  0.75247403,  3.25826483, -0.48329325],
              [ 0.39870151, -0.17244919, -0.48329325,  3.1017528 ]]).astype('complex')
# u, s, vh = np.linalg.svd(A, full_matrices=False)
# print(s)

b = np.array([0, 0, 0, 1]).T
x = np.linalg.solve(A, b)
print(x)

[-0.13618659+0.j  0.02853499+0.j  0.05535159+0.j  0.35011484+0.j]


In [6]:
def get_gate(A, n):    
    pauli_op = PauliSumOp(SparsePauliOp.from_operator(A))
    phi = Parameter('ϕ')
    evolution_op = (phi * pauli_op).exp_i() # exp(-iϕA)
    trotterized_op = PauliTrotterEvolution(trotter_mode=Suzuki(order=2, reps=1)).convert(evolution_op).bind_parameters({phi: np.pi/n})
    #----control---------
    gate = trotterized_op.to_circuit()
    # print(gate)
    gate.name = f"e^(i*A*pi/{n})"
    gate.label = f"e^(i*A*np.pi/{n})"
    gate = gate.to_gate().control()
    #---------------------
    return gate

In [7]:
n_b = 2
n_ = 20 #3 is an optimal
n_ancilla = 1
n_cl = 3
# quantum circuit initialization
qc = QuantumCircuit(n_b + n_ + n_ancilla, n_cl)
# b-vector state preparation
for i in range(n_b):
    qc.ry(np.pi/4, i)
for i in range(n_b, n_b+n_):
    qc.h(i)
qc.x(-1)
qc.barrier()
# Matrix exponentiation
for i in range(1, n_+1):
    gate = get_gate(A, 2**(i))
    qc.append(gate,[i+1, 0, 1])
qc.barrier()
# Phase estimation
for j in range(n_b + n_ - 1, n_b, -1):
    qc.h(j)
    for m in range(j - n_b):
        qc.crz(-np.pi/float(2**(j-m - n_b)), j, m+n_b)
qc.h(j-1)
qc.barrier()
# As I understood, we wncode ancilla qubit to be sure that result will be correct
for j in range(1, 1+n_):
    qc.cry(np.pi/(2**j), n_b+n_-j, n_b+n_)
qc.barrier()
# Inverse quantum Fourier transform
for j in range(n_b, n_b + n_):
    for m in range(j - n_b):
        qc.crz(np.pi/float(2**(j-m - n_b)), j, m+n_b)
    qc.h(j)
qc.h(n_b)
qc.barrier()
# Eigenvalues storing in the vecor b register
for i in range(n_, 0, -1):
    gate = get_gate(A, -(2**(i)))
    qc.append(gate,[i+1, 0, 1], )
qc.barrier()
# qubits measurement. I do not measure the ancilla qubit
qc.measure(-1, 2)
qc.measure(0, 0)
qc.measure(1, 1)

# qc.draw(output='mpl', style={'backgroundcolor': '#EEEEEE'})

In [ ]:
simulator = Aer.get_backend('qasm_simulator')
circ = transpile(qc, simulator)
shots = 2048
result = execute(qc, backend=simulator, shots=shots).result()


In [21]:
# np.sqrt(4 * (np.pi/4 ** 2)
(1/4)**2

0.0625

In [22]:
np.sqrt(0.0625)

0.25

In [25]:
np.arccos(0.25)

1.318116071652818

In [26]:
val = np.cos(np.pi/4)

0.7071067811865476

In [ ]:
#for each element in tensor:
el = val*val
#when reading:
out_val = np.sqrt(val)
#each tensor element of one qubit:
#[cos^2(alpha), sin^2(alpha)]
#then we have to do an sqrt and devide over 4 in case of two qubits (4 tensored elements)